In [1]:
import requests
import websocket
import threading
import time
import json
import queue

import pandas as pd
import numpy as np


In [2]:
url = 'https://api.binance.us/api/v3/depth?symbol=LINKUSDT&limit=1'

data = requests.get(url).json()

data

{'lastUpdateId': 248021168,
 'bids': [['10.87100000', '13.70000000']],
 'asks': [['10.88500000', '0.23000000']]}

In [3]:
def get_unix_time():
    return time.time_ns() // 1000

print(get_unix_time())

1722798110225980


In [4]:
class Client(threading.Thread):
    def __init__(self, url, exchange, interval):
        super().__init__()
        self.ws = websocket.WebSocketApp(
            url=url,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close,
            on_open=self.on_open
        )

        self.exchange = exchange
        self.interval = interval
        self.stop_event = threading.Event()

    def run(self):
        self.ws.run_forever(ping_interval = self.interval)

    def close(self):
        self.stop_event.set()
        if self.ws:
            self.ws.close()

    def on_message(self, ws, message):
        print(message)

    def on_error(self, ws, error):
        print(f"{self.exchange} Error: {error}")

    def on_close(self, ws, error_status, error_message):
        print(f"{self.exchange} Closed. Error Status: {error_status}. Error Message: {error_message}")

    def on_open(self, ws):
        print(f'Connected to {self.exchange}\n')

In [5]:
url1 = "wss://stream.binance.us:9443/ws/usdcusdt@depth10"
url2 = "wss://stream.binance.us:9443/ws/btcusdt@depth10"

ws_client_1 = Client(url1, "Binance", 100000)
ws_client_2 = Client(url2, "Binance", 100000)

ws_client_1.start()
ws_client_2.start()

Connected to Binance

Connected to Binance

{"lastUpdateId":2500013314,"bids":[["58747.62000000","0.00800000"],["58747.61000000","0.02700000"],["58739.73000000","0.93855000"],["58739.72000000","0.00465000"],["58732.90000000","0.12300000"],["58730.36000000","0.02821000"],["58715.56000000","0.35482000"],["58712.07000000","0.28856000"],["58712.06000000","0.00109000"],["58689.70000000","0.47631000"]],"asks":[["58805.29000000","0.00008000"],["58805.30000000","0.12300000"],["58857.85000000","0.20010000"],["58857.86000000","0.02700000"],["58860.64000000","0.28856000"],["58880.79000000","0.01254000"],["58880.80000000","0.47631000"],["58897.19000000","0.01043000"],["58897.20000000","0.47631000"],["58903.73000000","0.00032000"]]}
{"lastUpdateId":17677968,"bids":[["0.99980000","2014.00000000"],["0.99940000","640.00000000"],["0.99930000","258.00000000"],["0.99850000","2040.00000000"],["0.99800000","1020.00000000"],["0.99670000","301.00000000"],["0.99650000","1056.00000000"],["0.99640000","2700.000

In [6]:
ws_client_1.close()
ws_client_2.close()

Binance Closed. Error Status: None. Error Message: None
Binance Closed. Error Status: None. Error Message: None


In [8]:
class SymbolWatcher(Client):
    def __init__(self, symbol, depth, interval):
        url = f"wss://stream.binance.us:9443/ws/{symbol}@depth{depth}"
        self.message_buffer = queue.Queue()
        self.symbol = symbol
        super().__init__(url, "Binance", interval)
        print("watcher created successfully")

    def on_message(self, ws, message):
        self.message_buffer.put(message)
        print("message received")
    
    def write_to_file(self):
        with open(f"{self.symbol}.txt", 'a') as file:
            while not self.stop_event.is_set() or not self.message_buffer.empty():
                try:
                    message = self.message_buffer.get(timeout=1)
                    data = json.loads(message)
                    bids = data["bids"]
                    asks = data["asks"]
                    for i in range(min(len(bids), len(asks))):
                        entry = f"{data['lastUpdateId']}, {get_unix_time()}, {bids[i][0]}, {asks[i][0]}, {bids[i][1]}, {asks[i][1]}\n"
                        file.write(entry)
                        print("data logged")
                except queue.Empty:
                    continue
                except json.JSONDecodeError as e:
                    print(f"JSON decode error: {e}")
                except Exception as e:
                    print(f"Unexpected error: {e}")

In [9]:
ws_client_1 = SymbolWatcher("usdcusdt",10,10000)
ws_client_1_writer = threading.Thread(target=ws_client_1.write_to_file)

ws_client_1.start()
ws_client_1_writer.start()

watcher created successfully


Connected to Binance

message receiveddata logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged

message receiveddata logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged

message receiveddata logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged

message receiveddata logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged



In [10]:
ws_client_1.close()

Binance Closed. Error Status: None. Error Message: None
